In [ ]:
!pip install fuzzywuzzy


In [ ]:
import pandas as pd
import re
from collections import defaultdict
from fuzzywuzzy import fuzz
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk


nltk.download('stopwords')
nltk.download('wordnet')

df = pd.read_excel('/content/Gửi Huy.xlsx',sheet_name=1)

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))


In [ ]:
#Function to normalize keywords
def normalize_text(text):
  text = text.lower().strip() #convert+remove space
  text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
  words = text.split()
  words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words] #lemmatization & remove stopwords
  return ' '.join(words)

In [ ]:
#Normalize keywords
df['normalized_keywords'] = df['keyword'].astype(str).apply(normalize_text)

In [ ]:
#Merge similar keyword
keyword_groups = defaultdict(list)
for keyword in df["normalized_keywords"].unique():
  matched = False
  for group in keyword_groups:
    if fuzz.ratio(keyword, group) > 80:
      keyword_groups[group].append(keyword)
      matched = True
      break
  if not matched:
    keyword_groups[keyword].append(keyword)

In [ ]:
#Create a mapping
keyword_map = {variant: main for main, variants in keyword_groups.items() for variant in variants}
df["merged_keywords"] = df["normalized_keywords"].map(keyword_map)

In [ ]:
print(df.head())  # Print the first few rows of the DataFrame
print(df.columns) # Print the columns of the DataFrame

   keyword  searched count  search-to-watch normalized_keywords  \
0  youtube           37448            10544             youtube   
1      NHA           13348             7088                 nha   
2     NHAC           12853             7278                nhac   
3    anime           12849             7482               anime   
4      PHI           12713             5684                 phi   

  merged_keywords  
0         youtube  
1             nha  
2             nha  
3           anime  
4             phi  
Index(['keyword', 'searched count', 'search-to-watch', 'normalized_keywords',
       'merged_keywords'],
      dtype='object')


In [ ]:
if "Search-to-watch" not in df.columns:
    print("Column 'Searched_to_watch' is missing!")


Column 'Searched_to_watch' is missing!


In [ ]:
df_grouped = df.groupby("merged_keywords").agg({
    "searched count": "sum",  # Replace with the actual column name
    "Searched-to-watch": "sum"  # Replace with the actual column name
    }).reset_index()
    # ... rest of your code ...

KeyError: 'merged_keywords'

In [ ]:
df_grouped['CTR'] = (df_grouped['Searched_to_watch'] / df_grouped['Searched Count'].replace(0, np.nan)) * 100
df_grouped['CTR'].fillna(0, inplace=True)  # Replace NaN values with 0
df_grouped['CTR'] = df_grouped['CTR'].round(2)  # Round to 2 decimal places

NameError: name 'df_grouped' is not defined

In [ ]:
#Set filtering thresholds
MIN_SEARCH_VOLUME = 10  # Minimum searches to be considered
MIN_CTR = 5  # Minimum CTR (%) to keep a keyword

In [ ]:
#Apply filters
filtered_df = df_grouped[
    (df_grouped["Searched Count"] >= MIN_SEARCH_VOLUME) &
    (df_grouped["CTR"] >= MIN_CTR)
]


In [ ]:
#Save cleaned data
filtered_df.to_csv("filtered_data.csv", index=False)

print(f"Filtered dataset daved with {len(filtered_df)} keywords.")
print(filtered_df.head())

Filtered dataset daved with 9090 keywords.
  merged_keywords  Searched Count  Searched_to_watch         CTR
0                           52837              20909  252.699794
1               0              43                 14  307.142857
3            1 co              92                 30  306.666667
4        1 con vi              57                 34  167.647059
5              10             123                 58  212.068966


In [ ]:
import pandas as pd
import openpyxl # For working with existing Excel files
from openpyxl.utils.dataframe import dataframe_to_rows

# ... (code to generate the filtered_df DataFrame) ...

# Load the existing Excel file
existing_file_path = '/content/Gửi Huy.xlsx'
book = openpyxl.load_workbook(existing_file_path)

# Get the desired sheet (or create it if it doesn't exist)
new_sheet_name = 'NewFilteredData'  # Choose a name for the new sheet
sheet = book.create_sheet(title=new_sheet_name)

# Write the filtered DataFrame to the sheet, starting from a specific cell
start_row = 1  # Row to start writing data
start_col = 1  # Column to start writing data
for r in dataframe_to_rows(filtered_df, index=False, header=True):
    sheet.append(r)


# Save the changes to the existing file
book.save(existing_file_path)